In [11]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup


## Merge the Data

We'll be concatenating all the dataframes, droping any duplicated entries and observations with missing data

In [12]:
pd.read_csv('file_data_df_final.csv', index_col='timestamp').sort_index().iloc[[-1]].index

Int64Index([1355317620], dtype='int64', name='timestamp')

In [13]:
pd.read_csv('database_data.csv', index_col='timestamp').sort_index().iloc[[-1]].index

Int64Index([1351995420], dtype='int64', name='timestamp')

In [14]:
#html= pd.read_csv('data_from_html.csv', index_col='Timestamp').sort_index().iloc[[-1]].index
html= pd.read_csv('data_from_html.csv', index_col='Timestamp')
html = html.drop(columns =["Unnamed: 0.1",'Unnamed: 0'], axis =1)
html.columns = html.columns.str.lower()

html.to_csv('website_data.csv')

In [15]:
pd.read_csv('api_data.csv', index_col='Timestamp')

,rsi10,%k200,%k30,ema10
Timestamp,,,,
1325346600,NaN,NaN,NaN,NaN
1325350740,NaN,NaN,NaN,NaN
1325350800,NaN,NaN,NaN,NaN
1325391360,NaN,NaN,NaN,NaN
1325431680,NaN,NaN,NaN,NaN
...,...,...,...,...
1351989660,NaN,NaN,NaN,10.655024
1351991700,NaN,NaN,NaN,10.617747
1351994460,NaN,NaN,NaN,10.621793


In [16]:
df = pd.concat([
    pd.read_csv('api_data.csv', index_col='Timestamp'),
    pd.read_csv('file_data_df_final.csv', index_col='timestamp'),
    pd.read_csv('database_data.csv', index_col='timestamp'),
    pd.read_csv('website_data.csv', index_col='Timestamp'),
    pd.read_csv('signal.csv', index_col='timestamp')
    
])
df.sort_index()

,rsi10,%k200,%k30,ema10,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square,ma10,rsi30,%k10,ema200,ma200,rsi200,Unnamed: 0,signal
1325317920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1325317920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1325317920,NaN,NaN,NaN,NaN,4.39,4.39,4.39,4.39,0.455581,2.000,4.39,4.079757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1325317980,NaN,NaN,NaN,NaN,4.39,4.39,4.39,4.39,0.455581,2.000,4.39,4.079757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1325318040,NaN,NaN,NaN,NaN,4.39,4.39,4.39,4.39,0.455581,2.000,4.39,4.079757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355317380,NaN,NaN,NaN,NaN,13.34,13.34,13.34,13.34,3.900000,52.026,13.34,0.490000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1355317440,NaN,NaN,NaN,NaN,13.34,13.34,13.34,13.34,3.900000,52.026,13.34,0.490000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1355317500,NaN,NaN,NaN,NaN,13.34,13.34,13.34,13.34,3.900000,52.026,13.34,0.490000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1355317560,NaN,NaN,NaN,NaN,13.34,13.34,13.34,13.34,3.900000,52.026,13.34,0.490000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# so lets fill the null values (within the same column and the same id) 
# with any non null value for the same id
#df = df.fillna(method='ffill').fillna(method='bfill').sort_index()


In [18]:
# then keep only 1 row per id
df = df.groupby(df.index).first()
#df = df.drop(columns =['Unnamed: 0'], axis =1)
df.index.name = 'timestamp'

In [19]:
#df=df.dropna(how = 'all')
#df = df.dropna(subset = 'signal')
#df

## Create Train Dataset

In [20]:
df.to_csv("merge_data.csv")